In [1]:
%pip install langchain langchain-core langchain-community pypdf langchain_huggingface pinecone pinecone_text sentence-transformers cohere tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: mmh3
    Found existing installation: mmh3 5.2.0
    Uninstalling mmh3-5.2.0:
      Successfully uninstalled mm

In [3]:
from pinecone import Pinecone, ServerlessSpec
import os
os.environ["PINECONE_API_KEY"] = "pcsk_........."

pc = Pinecone()

index_name = "final-rag-trailer"

index = pc.Index(index_name)

In [4]:
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

In [5]:
from pinecone_text.sparse import BM25Encoder
bm25 = BM25Encoder.default()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import json

FILE_PATH = "/content/rbac_eval.jsonl"

records = []

with open(FILE_PATH, "r") as f:
    for line in f:
        records.append(json.loads(line))

print(f"Total records: {len(records)}")

print("\nFirst 5 records:\n")
for i, record in enumerate(records[:5], start=1):
    print(f"Record {i}:")
    print(json.dumps(record, indent=2))
    print("-" * 50)

Total records: 297

First 5 records:

Record 1:
{
  "role": "employee",
  "question": "How much monthly food allowance do junior employees receive?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0"
  ],
  "test_role": "hr"
}
--------------------------------------------------
Record 2:
{
  "role": "employee",
  "question": "Is meal card allowance tax exempt?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_0"
  ],
  "test_role": "hr"
}
--------------------------------------------------
Record 3:
{
  "role": "employee",
  "question": "What is the internet reimbursement limit for work from home?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_Benefits_parent_1"
  ],
  "test_role": "hr"
}
--------------------------------------------------
Record 4:
{
  "role": "employee",
  "question": "Does health insurance cover parents?",
  "relevant_chunk_ids": [
    "PAYROLL_&_BENEFITS_DOCUMENT_Employee_B

In [7]:
def run_rag(role: str, question: str):

    query_dense = emb.embed_query(question)
    query_sparse = bm25.encode_queries([question])[0]

    results = index.query(
        vector=query_dense,
        sparse_vector=query_sparse,
        top_k=5,
        include_metadata=True,
        filter={role: {"$eq": True}}
    )

    if not results.matches:
        return []

    filtered = [
        m for m in results.matches
        if m.score >= PINECONE_SCORE_THRESHOLD
    ]

    if not filtered:
        return []

    filtered.sort(key=lambda x: x.score, reverse=True)

    final = []
    for m in filtered[:TOP_K]:
        final.append({
            "doc_id": m.metadata.get("parent_id"),
            "chunk": m.metadata.get("text"),
            "pinecone_score": m.score
        })

    return final

In [11]:
import json
from typing import List, Dict

RBAC_FILE = "/content/rbac_eval.jsonl"
TOP_K = 5
PINECONE_SCORE_THRESHOLD = 0.5

def load_data(path: str) -> List[Dict]:
    data = []
    with open(path) as f:
        for line in f:
            data.append(json.loads(line))
    return data


def get_doc_type(doc_id):
    if doc_id.startswith("PAYROLL_&_BENEFITS_DOCUMENT"):
        return "payroll"
    if doc_id.startswith("hr_policy_documents"):
        return "hr_policy"
    if doc_id.startswith("EMPLOYEE_LIFECYCLE_DOCUMENT"):
        return "lifecycle"
    if doc_id.startswith("COMPANY-SPECIFIC_DOCUMENT"):
        return "company"
    return "unknown"


def is_violation(doc_id, test_role):
    dtype = get_doc_type(doc_id)


    access_rules = {
        "payroll": ["employee", "manager"],
        "hr_policy": ["employee"],
        "lifecycle": ["manager"],
        "company": ["hr"]
    }

    allowed_roles = access_rules.get(dtype, [])
    return test_role not in allowed_roles


def evaluate_rbac():
    data = load_data(RBAC_FILE)

    total = 0
    violations = 0
    critical = 0
    medium = 0

    failure_cases = []

    for row in data:
        question = row["question"]
        test_role = row["test_role"]
        expected_doc = row["relevant_chunk_ids"][0]

        total += 1

        retrieved = run_rag(test_role, question)

        for rank, r in enumerate(retrieved, start=1):
            doc_id = r["doc_id"]

            if is_violation(doc_id, test_role):
                violations += 1

                if rank == 1:
                    critical += 1
                else:
                    medium += 1

                failure_cases.append({
                    "question": question,
                    "test_role": test_role,
                    "leaked_doc": doc_id,
                    "rank": rank,
                    "doc_type": get_doc_type(doc_id)
                })
                break

    violation_rate = violations / total if total else 0

    print("\n🔐 RBAC SECURITY EVALUATION")
    print("="*40)
    print(f"Total attack tests   : {total}")
    print(f"Violations found    : {violations}")
    print(f"Violation rate      : {violation_rate:.4f}")
    print(f"Critical (Rank 1)   : {critical}")
    print(f"Medium (Rank >1)    : {medium}")
    print("="*40)

    if failure_cases:
        print("\n--- SAMPLE LEAK CASES ---")
        for case in failure_cases[:5]:
            print(json.dumps(case, indent=2))
    else:
        print("\n✅ No RBAC leaks detected")

    return {
        "total_tests": total,
        "violations": violations,
        "violation_rate": violation_rate,
        "critical": critical,
        "medium": medium
    }

if __name__ == "__main__":
    evaluate_rbac()


🔐 RBAC SECURITY EVALUATION
Total attack tests   : 297
Violations found    : 0
Violation rate      : 0.0000
Critical (Rank 1)   : 0
Medium (Rank >1)    : 0

✅ No RBAC leaks detected
